In [1]:
%cd /app/
import tensorflow as tf
from pathlib import Path
from src.model import create_model
import tensorflow_addons as tfa
for gpu in tf.config.list_physical_devices('GPU'):
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)
import pickle
import numpy as np
import matplotlib.pyplot as plt  
from src.dataset import Dataset
# tf.debugging.set_log_device_placement(True)

/app
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [2]:
old_model_path = Path('../checkpoints') /'20200330-183949'
new_model_path = Path('models/20200405-194839')

In [3]:
labels = Path('labels.txt').read_text()
labels = labels.split('\n')
output_shape = len(labels)
latest = tf.train.latest_checkpoint(str(new_model_path))
model = create_model((224,224, 3), output_shape)

31784960/31782304 [==============================] - 2s 0us/step


In [4]:
ds_train, ds_val, class_weight, le, IMAGE_SIZE, N_CLASSES = Dataset.read(batch_size=22)

In [6]:
model.load_weights(latest)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 224, 224, 3)       7         
_________________________________________________________________
random_flip (RandomFlip)     (None, 224, 224, 3)       0         
_________________________________________________________________
random_translation (RandomTr (None, 224, 224, 3)       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 224, 224, 3)       0         
_________________________________________________________________
random_contrast (RandomContr (None, 224, 224, 3)       0         
_________________________________________________________________
efficientnet-b2 (Model)      (None, 7, 7, 1408)        7768562   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1408)              0

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=5)])

In [10]:
model.evaluate(ds_val.take(300))

300/300 [==============================] - 27s 90ms/step - loss: 1.8151 - accuracy: 0.6379 - top_k_categorical_accuracy: 0.8423


[1.8151389360427856, 0.6378787755966187, 0.8422726988792419]

In [ ]:
#model.save('great_model')

In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [12]:
# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
tflite_model_file = Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)


35840956